In [30]:
GPU_device_id = str(6)
model_id_save_as = 'daepretrain-easy-final'
architecture_id = 'final-models/learningcurve-dnn-easy-final-features'
model_class_id = 'DAE'
training_dataset_id = '../dataset_generation/hyperparametersearch_dataset_200keV_easy_log10time_1000.npy'
difficulty_setting = 'easy'

earlystop_patience = 10
num_epochs = 2000

In [2]:
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = GPU_device_id


from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
from random import choice

from numpy.random import seed
seed(5)
from tensorflow import set_random_seed
set_random_seed(5)



#### Import model, training function 

In [3]:
from annsa.model_classes import build_dae_model, compile_model, f1, mean_normalized_kl_divergence
from annsa.load_dataset import load_easy, load_full, dataset_to_spectrakeys
from annsa.load_pretrained_network import load_features

Using TensorFlow backend.


## Training Data Construction

In [4]:
training_dataset = np.load(training_dataset_id, allow_pickle=True)
training_source_spectra, training_background_spectra, training_keys = dataset_to_spectrakeys(training_dataset,
                                                                                             sampled=False,
                                                                                             separate_background=True)

## Load Model

In [5]:
model_features = load_features(architecture_id)
model_features.loss = tf.keras.losses.mean_squared_error
model_features.optimizer = tf.keras.optimizers.Adam
model_features.learning_rate = model_features.learining_rate
model_features.input_dim = 1024

In [6]:
dae_features = model_features.to_dae_model_features()
dae_features.metrics = ['mse']

## Train network

# Scale input data

In [8]:
training_input = np.random.poisson(training_source_spectra+training_background_spectra)
training_output = training_source_spectra

training_input = dae_features.scaler.transform(training_input)
training_output = dae_features.scaler.transform(training_output)


/home/ubuntu/anaconda3/envs/tensorflow_p36_update/lib/python3.6/site-packages/sklearn/preprocessing/_function_transformer.py:161: RuntimeWarning: invalid value encountered in sqrt
  **(kw_args if kw_args else {}))


In [9]:
earlystop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_mse',
    patience=earlystop_patience,
    mode='min',
    restore_best_weights=True)

csv_logger = tf.keras.callbacks.CSVLogger('./final-models-keras/'+model_id_save_as+'.log')

In [10]:
mlb=LabelBinarizer()
model = compile_model(
            build_dae_model,
            dae_features)

output = model.fit(
    x=training_input,
    y=training_output,
    batch_size=model_features.batch_size,
    validation_split=0.1,
    epochs=500,
    verbose=1,
    shuffle=True,
    callbacks=[earlystop_callback, ],
)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 27000 samples, validate on 3000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/500
27000/27000 [==============================] - 1s 30us/sample - loss: 23.3045 - mean_squared_error: 23.3045 - val_loss: 14.8061 - val_mean_squared_error: 14.8061
Epoch 2/500
27000/27000 [==============================] - 0s 16us/sample - loss: 10.4501 - mean_squared_error: 10.4501 - val_loss: 10.2125 - val_mean_squared_error: 10.2125
Epoch 3/500
27000/27000 [==============================] - 0s 16us/sample - loss: 7.8598 - mean_squared_error: 7.8598 - val_loss: 7.4539 - val_mean_squared_error: 7.4539
Epoch 4/500
27000/27000 [==============================] - 0s 16us/sample - loss: 6.1130 - mean_squared_error: 6.1130 - val_loss: 5.8122 - val_mean_squared_error: 5.8122
Epoch 5/500
27000/27000 [===

27000/27000 [==============================] - 0s 16us/sample - loss: 2.0989 - mean_squared_error: 2.0989 - val_loss: 2.7146 - val_mean_squared_error: 2.7146
Epoch 18/500
27000/27000 [==============================] - 0s 16us/sample - loss: 2.0326 - mean_squared_error: 2.0326 - val_loss: 2.7754 - val_mean_squared_error: 2.7754
Epoch 19/500
27000/27000 [==============================] - 0s 16us/sample - loss: 1.9581 - mean_squared_error: 1.9581 - val_loss: 2.6273 - val_mean_squared_error: 2.6273
Epoch 20/500
27000/27000 [==============================] - 0s 16us/sample - loss: 1.8862 - mean_squared_error: 1.8862 - val_loss: 2.5629 - val_mean_squared_error: 2.5629
Epoch 21/500
27000/27000 [==============================] - 0s 16us/sample - loss: 1.8356 - mean_squared_error: 1.8356 - val_loss: 2.4512 - val_mean_squared_error: 2.4512
Epoch 22/500
27000/27000 [==============================] - 0s 16us/sample - loss: 1.7898 - mean_squared_error: 1.7898 - val_loss: 2.4216 - val_mean_squared_e

27000/27000 [==============================] - 0s 16us/sample - loss: 1.4894 - mean_squared_error: 1.4894 - val_loss: 2.1102 - val_mean_squared_error: 2.1102
Epoch 37/500
27000/27000 [==============================] - 0s 16us/sample - loss: 1.4680 - mean_squared_error: 1.4680 - val_loss: 2.1258 - val_mean_squared_error: 2.1258
Epoch 38/500
27000/27000 [==============================] - 0s 16us/sample - loss: 1.4756 - mean_squared_error: 1.4756 - val_loss: 2.1405 - val_mean_squared_error: 2.1405
Epoch 39/500
27000/27000 [==============================] - 0s 16us/sample - loss: 1.4623 - mean_squared_error: 1.4623 - val_loss: 2.1126 - val_mean_squared_error: 2.1126
Epoch 40/500
27000/27000 [==============================] - 0s 16us/sample - loss: 1.4385 - mean_squared_error: 1.4385 - val_loss: 2.0660 - val_mean_squared_error: 2.0660
Epoch 41/500
27000/27000 [==============================] - 0s 16us/sample - loss: 1.4367 - mean_squared_error: 1.4367 - val_loss: 2.0927 - val_mean_squared_e

27000/27000 [==============================] - 0s 16us/sample - loss: 1.3204 - mean_squared_error: 1.3204 - val_loss: 1.9966 - val_mean_squared_error: 1.9966
Epoch 56/500
27000/27000 [==============================] - 0s 16us/sample - loss: 1.3128 - mean_squared_error: 1.3128 - val_loss: 2.0299 - val_mean_squared_error: 2.0299
Epoch 57/500
27000/27000 [==============================] - 0s 16us/sample - loss: 1.2947 - mean_squared_error: 1.2947 - val_loss: 2.0531 - val_mean_squared_error: 2.0531
Epoch 58/500
27000/27000 [==============================] - 0s 16us/sample - loss: 1.2828 - mean_squared_error: 1.2828 - val_loss: 2.0085 - val_mean_squared_error: 2.0085
Epoch 59/500
27000/27000 [==============================] - 0s 16us/sample - loss: 1.3001 - mean_squared_error: 1.3001 - val_loss: 2.0039 - val_mean_squared_error: 2.0039
Epoch 60/500
27000/27000 [==============================] - 0s 16us/sample - loss: 1.2834 - mean_squared_error: 1.2834 - val_loss: 2.0742 - val_mean_squared_e

27000/27000 [==============================] - 0s 16us/sample - loss: 1.1997 - mean_squared_error: 1.1997 - val_loss: 2.0556 - val_mean_squared_error: 2.0556
Epoch 75/500
27000/27000 [==============================] - 0s 16us/sample - loss: 1.1772 - mean_squared_error: 1.1772 - val_loss: 2.0603 - val_mean_squared_error: 2.0603
Epoch 76/500
27000/27000 [==============================] - 0s 16us/sample - loss: 1.1804 - mean_squared_error: 1.1804 - val_loss: 2.0743 - val_mean_squared_error: 2.0743
Epoch 77/500
27000/27000 [==============================] - 0s 16us/sample - loss: 1.1720 - mean_squared_error: 1.1720 - val_loss: 1.9942 - val_mean_squared_error: 1.9942
Epoch 78/500
27000/27000 [==============================] - 0s 16us/sample - loss: 1.1754 - mean_squared_error: 1.1754 - val_loss: 1.9968 - val_mean_squared_error: 1.9968
Epoch 79/500
27000/27000 [==============================] - 0s 16us/sample - loss: 1.1598 - mean_squared_error: 1.1598 - val_loss: 2.0387 - val_mean_squared_e

...

27000/27000 [==============================] - 0s 16us/sample - loss: 0.8949 - mean_squared_error: 0.8949 - val_loss: 2.1816 - val_mean_squared_error: 2.1816
Epoch 417/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.8888 - mean_squared_error: 0.8888 - val_loss: 2.2241 - val_mean_squared_error: 2.2241
Epoch 418/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.8904 - mean_squared_error: 0.8904 - val_loss: 2.1524 - val_mean_squared_error: 2.1524
Epoch 419/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.9017 - mean_squared_error: 0.9017 - val_loss: 2.1821 - val_mean_squared_error: 2.1821
Epoch 420/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.8900 - mean_squared_error: 0.8900 - val_loss: 2.1762 - val_mean_squared_error: 2.1762
Epoch 421/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.8953 - mean_squared_error: 0.8953 - val_loss: 2.1629 - val_mean_squa

27000/27000 [==============================] - 0s 16us/sample - loss: 0.8898 - mean_squared_error: 0.8898 - val_loss: 2.1559 - val_mean_squared_error: 2.1559
Epoch 436/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.8894 - mean_squared_error: 0.8894 - val_loss: 2.1729 - val_mean_squared_error: 2.1729
Epoch 437/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.8861 - mean_squared_error: 0.8861 - val_loss: 2.1802 - val_mean_squared_error: 2.1802
Epoch 438/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.8879 - mean_squared_error: 0.8879 - val_loss: 2.1778 - val_mean_squared_error: 2.1778
Epoch 439/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.8924 - mean_squared_error: 0.8924 - val_loss: 2.1704 - val_mean_squared_error: 2.1704
Epoch 440/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.8852 - mean_squared_error: 0.8852 - val_loss: 2.1318 - val_mean_squa

27000/27000 [==============================] - 0s 17us/sample - loss: 0.8821 - mean_squared_error: 0.8821 - val_loss: 2.2025 - val_mean_squared_error: 2.2025
Epoch 455/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.8791 - mean_squared_error: 0.8791 - val_loss: 2.2043 - val_mean_squared_error: 2.2043
Epoch 456/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.8807 - mean_squared_error: 0.8807 - val_loss: 2.1377 - val_mean_squared_error: 2.1377
Epoch 457/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.8903 - mean_squared_error: 0.8903 - val_loss: 2.1441 - val_mean_squared_error: 2.1441
Epoch 458/500
27000/27000 [==============================] - 0s 17us/sample - loss: 0.8855 - mean_squared_error: 0.8855 - val_loss: 2.2081 - val_mean_squared_error: 2.2081
Epoch 459/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.9034 - mean_squared_error: 0.9034 - val_loss: 2.1759 - val_mean_squa

27000/27000 [==============================] - 0s 16us/sample - loss: 0.8826 - mean_squared_error: 0.8826 - val_loss: 2.1530 - val_mean_squared_error: 2.1530
Epoch 474/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.8710 - mean_squared_error: 0.8710 - val_loss: 2.1670 - val_mean_squared_error: 2.1670
Epoch 475/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.8812 - mean_squared_error: 0.8812 - val_loss: 2.1798 - val_mean_squared_error: 2.1798
Epoch 476/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.8746 - mean_squared_error: 0.8746 - val_loss: 2.1831 - val_mean_squared_error: 2.1831
Epoch 477/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.8732 - mean_squared_error: 0.8732 - val_loss: 2.1511 - val_mean_squared_error: 2.1511
Epoch 478/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.8796 - mean_squared_error: 0.8796 - val_loss: 2.1619 - val_mean_squa

27000/27000 [==============================] - 0s 16us/sample - loss: 0.8706 - mean_squared_error: 0.8706 - val_loss: 2.1555 - val_mean_squared_error: 2.1555
Epoch 493/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.8784 - mean_squared_error: 0.8784 - val_loss: 2.1989 - val_mean_squared_error: 2.1989
Epoch 494/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.8699 - mean_squared_error: 0.8699 - val_loss: 2.1401 - val_mean_squared_error: 2.1401
Epoch 495/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.8706 - mean_squared_error: 0.8706 - val_loss: 2.1502 - val_mean_squared_error: 2.1502
Epoch 496/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.8822 - mean_squared_error: 0.8822 - val_loss: 2.1380 - val_mean_squared_error: 2.1380
Epoch 497/500
27000/27000 [==============================] - 0s 16us/sample - loss: 0.8827 - mean_squared_error: 0.8827 - val_loss: 2.1482 - val_mean_squa

In [31]:
model.save('./final-models-keras/'+model_id_save_as+'.hdf5')